# Reproducing Shlomi et al. 2009 Figure 2 and 3
Original publication: http://doi.org/10.1038/msb.2009.22

Thierry D.G.A. Mondeel, Vivian Ogundipe & Hans V. Westerhoff <br>
Unversity of Amsterdam  <br>
October 2017

For details on our approach and implementation see the ReScience manuscript.

## Import the model and set the list of potential biomarkers
Shlomi et al. used Recon 1. The Recon 1 version used here has been downloaded from the BiGG database: http://bigg.ucsd.edu/models/RECON1. Figure 2 only checks for the proteinogenic amino acids as possible biomarkers.

In [17]:
import cobra
from findBiomarkers import findBiomarkers

import pandas as pd
pd.set_option('display.max_colwidth', -1)

from IPython.display import clear_output
import json

# import Recon 1
M = cobra.io.load_json_model("../data/RECON1.json")

# All exchange reactions for amino acids
essentialAA = ['EX_his__L_e','EX_ile__L_e','EX_leu__L_e','EX_lys__L_e','EX_met__L_e','EX_phe__L_e','EX_thr__L_e','EX_trp__L_e','EX_val__L_e']
nonEssentialAA = ['EX_cys__L_e','EX_glu__L_e','EX_tyr__L_e','EX_ala__L_e','EX_asp__L_e','EX_gly_e','EX_arg__L_e','EX_gln__L_e','EX_pro__L_e','EX_ser__L_e','EX_asn__L_e']
rxnsOfInterest = [essentialAA,nonEssentialAA]
rxnsOfInterest = [ item for sublist in rxnsOfInterest for item in sublist ]

### Set the medium of Recon 1
The publication does not explicitly discuss the medium. We found that inward fluxes bounded to -1 and outward fluxes bounded to 1000 allow reproducibility of the results

In [18]:
# Note: exchange reactions are pseudo-reactions of the form: 'X_e <=> '
exchanges = [ rxn for rxn in M.reactions if len(rxn.products) == 0]

# all inward fluxes to -1
# all outward fluxes to 1000
for rxn in exchanges:
    rxn.lower_bound = -10
for rxn in exchanges:
        rxn.upper_bound = 1000
        
M.medium;
len(exchanges)

430

## Read in the IEM data from Shlomi et al. supplemental information
Shlomi et al. provided a supplementary Excel table linking IEMs to genes and reactions in Recon 1. 

For biological completeness, we chose to read the genes associated to the IEMs for the Excel table provided by Shlomi et al. and subsequently link these to reactions through Recon 1. 

We filter out all the information related to the 17 amino acid disorders considered in their Figure 2.

In [19]:
IEMinfo = pd.read_excel('../data/IEM_compendium.xlsx', sheet_name='mappedIEMs supple table', header=0)

# note the double entry for Hypermethioninemia
figure2IEMs = ['S-Adenosylhomocysteine hydrolase', 'Alkaptonuria','Arginase deficiency','Cystinuria',
               'Lysinuric protein intolerance','Glutamate formiminotransferase deficiency',
              'Histidinemia','Homocystinuria','Hyperprolinemia type I','maple syrup urine disease',
              'Methionine adenosyltransferase deficiency','methylmalonic acidemia (MMA)','phenylketonuria',
              'Tetrahydrobiopterin deficiency','Tyrosinemia type 1','Tyrosinemia type III',
              'Glycine encephalopathy/ nonketotic hyperglycinemia / NKH']

# Filter out the information related to Figure 2
IEMinfo = IEMinfo[IEMinfo['disease (Mapped IEMs)'].isin(figure2IEMs)]

# make a dictionary: IEM -> Entrez gene ID and another one for: IEM -> reactions in Recon 1 
IEMdict = dict(zip(IEMinfo['disease (Mapped IEMs)'],IEMinfo['EntrezGeneID'] )) 
IEMdict_rxns = dict(zip(IEMinfo['disease (Mapped IEMs)'],IEMinfo['corresponding reaction in Recon1'] ))   
        
# update IEMdict to map to Recon 1 gene IDs not to Entrez
for iem in IEMdict:
    genelist = IEMdict[iem]
    
    if type(genelist) is not list: # in case there is one gene linked
        genelist = [genelist]

    # Recon 1 used Entrez IDs but appends things like .1 Look for the substrings
    IEMdict[iem] = list(set([ x for x in [gene.id for gene in M.genes] for y in genelist if x.find(str(y)+'_') != -1 ]))
    
    # IEMdict_rxns is a string with potentially multiple reactions. Turn into a list
    IEMdict_rxns[iem] = IEMdict_rxns[iem].split(',') 
    IEMdict_rxns[iem] = [str(x).strip() for x in IEMdict_rxns[iem]] # clean strings


## About the 4 IEMs for which we had reproducibility problems: 

We had issues with: S-Adenosylhomocysteine hydrolase, Methionine adenosyltransferase deficiency, Tetrahydrobiopterin deficiency (Phenylketonuria II) and Histidinemia. 

The first two have no entries in the Excel sheet. However, on the left of Figure 2 we can extract the enzyme directly, which seem to be AHCY (Entrez: 191) and MAT1a (Entrez: 4143) respectively. See:
- AHCY: https://www.ncbi.nlm.nih.gov/gene/?term=191
- MAT1a: https://www.ncbi.nlm.nih.gov/gene/?term=4143

For PKU II we originally found tyrosine whereas Shlomi et al. do not. However in the excel file multiple genes are linked to this IEM whereas Figure 2 on the left refers only to QDPR (5860). Limiting our simulation to QDPR returned the same results as in the publication. See: https://www.omim.org/entry/261630?search=261630&highlight=261630

The Excel file lists the reaction HISTD for Histidinemia. However, this should be HISTDr in Recon 1. The gene with ID 3034_AT1 refers to that reaction.

We update the IEMdict_rxns because this dictionary is what we use in the calculation below.

IEMdict_rxns entries before our changes

In [20]:
print(#IEMdict_rxns['S-Adenosylhomocysteine hydrolase'], # THIS ONE DOESN'T EXIST IN THE EXCEL FILE
    #IEMdict_rxns['Methionine adenosyltransferase deficiency'], # THIS ONE DOESN'T EXIST IN THE EXCEL FILE
    IEMdict_rxns['Tetrahydrobiopterin deficiency'],
    IEMdict_rxns['Histidinemia'])

['GTPCI', 'GTPCIn', 'THBPT4ACAMDASE', 'PTHPS', 'PTHPSn', 'DHPR'] ['HISD']


Below we show that without our changes the PKU II (tetrahydrobiopterin deficiency) does not agree with Shlomi et al.

In [21]:
findBiomarkers(M,rxnsOfInterest,IEMdict_rxns['Tetrahydrobiopterin deficiency'])

Interpreting mode as IEMrxn
Modifications will be performed on the following reactions:
                                                     Description                                    Reaction
GTPCI           GTP cyclohydrolase I                              gtp_c + h2o_c --> ahdt_c + for_c + h_c    
GTPCIn          GTP cyclohydrolase I, nuclear                     gtp_n + h2o_n --> ahdt_n + for_n + h_n    
THBPT4ACAMDASE  Tetrahydrobiopterin-4a-carbinolamine dehydratase  thbpt4acam_c --> dhbpt_c + h2o_c          
PTHPS           6-pyruvoyltetrahydropterin synthase               ahdt_c --> 6pthp_c + pppi_c               
PTHPSn          6-pyruvoyltetrahydropterin synthase, nuclear      ahdt_n --> 6pthp_n + pppi_n               
DHPR            6,7-dihydropteridine reductase                    dhbpt_c + h_c + nadh_c --> nad_c + thbpt_c



forward FVA could not be solved. Continuing without the forward interval.
backward FVA could not be solved. Continuing without the backward interval.
No healthy interval could be calculated
Empty wild-type and mutant FVA intervals. Skipping to next reaction.
forward FVA could not be solved. Continuing without the forward interval.
backward FVA could not be solved. Continuing without the backward interval.
No healthy interval could be calculated
Empty wild-type and mutant FVA intervals. Skipping to next reaction.
forward FVA could not be solved. Continuing without the forward interval.
backward FVA could not be solved. Continuing without the backward interval.
No healthy interval could be calculated
Empty wild-type and mutant FVA intervals. Skipping to next reaction.
forward FVA could not be solved. Continuing without the forward interval.
backward FVA could not be solved. Continuing without the backward interval.
No healthy interval could be calculated
Empty wild-type and mutant FVA in

,Name,Prediction,WT,Mutant,Score
ID,,,,,
tyr__L_e,L-Tyrosine,Reduced,"[-10.0, 10.0]","[-10.0, 0.0]",1.0


Here we actually manualy update the reactions linked to the 4 problematic IEMs.

In [22]:
IEMdict_rxns['S-Adenosylhomocysteine hydrolase'] = [str(r.id) for r in M.genes.get_by_id('191_AT1').reactions]
IEMdict_rxns['Methionine adenosyltransferase deficiency'] = [str(r.id) for r in M.genes.get_by_id('4143_AT1').reactions]
IEMdict_rxns['Tetrahydrobiopterin deficiency'] = [str(r.id) for r in M.genes.get_by_id('5860_AT1').reactions]
IEMdict_rxns['Histidinemia'] = [str(r.id) for r in M.genes.get_by_id('3034_AT1').reactions]

IEMdict_rxns entries AFTER our changes

In [23]:
print(IEMdict_rxns['S-Adenosylhomocysteine hydrolase'], # THIS ONE DOESN'T EXIST IN THE EXCEL FILE
    IEMdict_rxns['Methionine adenosyltransferase deficiency'], # THIS ONE DOESN'T EXIST IN THE EXCEL FILE
     IEMdict_rxns['Tetrahydrobiopterin deficiency'],
      IEMdict_rxns['Histidinemia'])

['AHCi', 'SEAHCYSHYD'] ['METAT', 'SELMETAT'] ['DHPR'] ['HISDr']


Show that PKU II is now in agreement with the predictions in Figure 2.

In [24]:
findBiomarkers(M,rxnsOfInterest,IEMdict_rxns['Tetrahydrobiopterin deficiency'])

Interpreting mode as IEMrxn
Modifications will be performed on the following reactions:
                         Description                                    Reaction
DHPR  6,7-dihydropteridine reductase  dhbpt_c + h_c + nadh_c --> nad_c + thbpt_c

0 low confidence biomarkers with scores below the threshold were                 found


,Name,Prediction,WT,Mutant,Score
ID,,,,,


## Figure 2: Calculate all biomarkers for all IEMs in the figure

The cell below should reproduce the exact method as Shlomi describes it for the list of IEMs presented in Figure 2 in the original publication. We get the genes from their Excel sheet, find them in RECON1, find their associated reactions and give that to findBiomarkers(). The default settings of the function are set to reproduce Shlomi's method.

In [25]:
model = M.copy()

bmPredictions = {}
count = 0

IEMlistToCalc = figure2IEMs

# init the dataframe
summary_df = pd.DataFrame(columns=['IEM','Affected reactions','Elevated','Reduced','Elevated sub-threshold','Reduced sub-threshold'])

for iem in IEMlistToCalc:
    clear_output()
    print(iem)
    print('Progress: {}%'.format((count*1.0)/len(IEMlistToCalc)*100))
    
    # predict biomarkers    
    print(IEMdict_rxns[iem])
    predictionTable = findBiomarkers(model,rxnsOfInterest,IEMdict_rxns[iem],threshold=0)
    
    # save pandas table in dictionary
    bmPredictions[iem] = predictionTable.to_string()
    
    # save new row in summary_df
    sign_up = [predictionTable.Name.values[i] for i in range(len(predictionTable)) if predictionTable.iloc[i]['Score'] >= 0.1 
               and predictionTable.iloc[i]['Prediction'] in ['Elevated','H.C. Elevated']]
    insign_up = [predictionTable.Name.values[i] for i in range(len(predictionTable)) if predictionTable.iloc[i]['Score'] < 0.1 
                and predictionTable.iloc[i]['Prediction'] in ['Elevated','H.C. Elevated']]
    sign_down = [predictionTable.Name.values[i] for i in range(len(predictionTable)) if predictionTable.iloc[i]['Score'] >= 0.1 
               and predictionTable.iloc[i]['Prediction'] in ['Reduced','H.C. Reduced']]
    insign_down = [predictionTable.Name.values[i] for i in range(len(predictionTable)) if predictionTable.iloc[i]['Score'] < 0.1 
                and predictionTable.iloc[i]['Prediction'] in ['Reduced','H.C. Reduced']]
    d = {'IEM':iem,'Elevated':sign_up,'Reduced':sign_down,'Elevated sub-threshold':insign_up,'Reduced sub-threshold':insign_down,
        'Affected reactions':IEMdict_rxns[iem]}
         #'Affected reactions':list(set([r.id for g in IEMdict[iem] for r in model.genes.get_by_id(g).reactions ]))}
    summary_df = summary_df.append(d,ignore_index=True)
    
    count = count + 1
    
# save result as csv and json
summary_df.to_csv('../data/predicted_biomarkers_figure2_default.csv')
    
with open('../data/predicted_biomarkers_figure2_default.json','w') as f: 
    f.write(json.dumps(bmPredictions, indent=4))
  
clear_output()
print("Completed calculation.")

Completed calculation.


### Export predictions to Excel

In [26]:
summary_df = pd.DataFrame.from_csv('../data/predicted_biomarkers_figure2_default.csv')

summary_df.set_index(['IEM'],inplace=True)

# clean the dataframe content
import ast
for i in range(len(summary_df)):
    for col in ['Affected reactions','Elevated','Reduced','Elevated sub-threshold','Reduced sub-threshold']:
        l = ast.literal_eval(summary_df.iloc[i][col])
        if type(l) == list and len(l) == 0:
            summary_df.iloc[i][col] = ''
        else:
            if len(l) > 1:
                summary_df.iloc[i][col] = ', '.join([str(x) for x in l])
            else:
                summary_df.iloc[i][col] = str(l[0]) 
                

# EXCEL PART
writer = pd.ExcelWriter('../data/Shlomi_fig2_results_summarized.xlsx', engine='xlsxwriter')
workbook = writer.book

format_null = workbook.add_format({'text_wrap': True,'align':'left','font_size':10})

summary_df.to_excel(writer,sheet_name='Shlomi_Fig2_reproduction', index=False)
worksheet = writer.sheets['Shlomi_Fig2_reproduction']

# freeze first row and first column
worksheet.freeze_panes(1, 1)

# Formatting
for row in range(len(summary_df.index)):
    worksheet.set_row(row + 1, None, format_null)
        
# set column widths
worksheet.set_column('A:B', 100) 
worksheet.set_column('C:F', 60) 

writer.save()

summary_df.to_excel('../data/Shlomi_fig2_results_summarized.xlsx')                
                

### Print the table we use in the manuscript
# in the publication we usually do not show the below threshold hits so remove these columns                
summary_df[['Affected reactions','Elevated','Reduced']]

/Users/thierry/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


,Affected reactions,Elevated,Reduced
IEM,,,
S-Adenosylhomocysteine hydrolase,"AHCi, SEAHCYSHYD",L-Methionine,L-Cysteine
Alkaptonuria,HGNTOR,L-Tyrosine,
Arginase deficiency,ARGN,,
Cystinuria,"CYSTSERex, SERLYSNaex",,
Lysinuric protein intolerance,SERLYSNaex,,
Glutamate formiminotransferase deficiency,"FTCD, GluForTx",L-Histidine,
Histidinemia,HISDr,L-Histidine,
Homocystinuria,"CYSTS, MTHFR3, METS",L-Methionine,L-Cysteine
Hyperprolinemia type I,"PRO1xm, PROD2m",,


### Show detailed results of computation

In [27]:
import json
from pprint import pprint

with open('../data/predicted_biomarkers_figure2_default.json') as data_file:    
    data = json.load(data_file)

for iem in data.keys():
    print(iem)
    print(data[iem])
    print()

S-Adenosylhomocysteine hydrolase
                  Name Prediction                WT            Mutant  Score
ID                                                                          
met__L_e  L-Methionine  Elevated   [-10.0, 0.0]      [-0.0, 0.0]       1.000
cys__L_e  L-Cysteine    Reduced    [-10.0, 40.0]     [-10.0, 30.0]     0.250
gln__L_e  L-Glutamine   Reduced    [-10.0, 808.333]  [-10.0, 775.833]  0.040
asn__L_e  L-Asparagine  Reduced    [-10.0, 805.902]  [-10.0, 775.833]  0.037

Alkaptonuria
                     Name Prediction                WT            Mutant  Score
ID                                                                             
tyr__L_e  L-Tyrosine       Elevated   [-10.0, 9.0]      [-10.0, 10.0]     0.100
asn__L_e  L-Asparagine     Elevated   [-10.0, 805.261]  [-10.0, 805.902]  0.001
gln__L_e  L-Glutamine      Elevated   [-10.0, 807.833]  [-10.0, 808.333]  0.001
ile__L_e  L-Isoleucine     Unchanged  [-10.0, 0.0]      [-10.0, 0.0]      0.000
ser__L_e  L

## Figure 3: Detailed look at AHCY and CBS
For Figure 3 the results are also included in the output above. However, we repeat it here for clarity.

In [28]:
findBiomarkers(M,rxnsOfInterest,IEMdict_rxns['S-Adenosylhomocysteine hydrolase'],threshold=0.1)

Interpreting mode as IEMrxn
Modifications will be performed on the following reactions:
                                        Description                                 Reaction
AHCi        Adenosylhomocysteinase                   ahcys_c + h2o_c <=> adn_c + hcys__L_c  
SEAHCYSHYD  Se-Adenosylselenohomocysteine hydrolase  h2o_c + seahcys_c --> adn_c + selhcys_c



forward FVA could not be solved. Continuing without the forward interval.
backward FVA could not be solved. Continuing without the backward interval.
No healthy interval could be calculated
Empty wild-type and mutant FVA intervals. Skipping to next reaction.

2 low confidence biomarkers with scores below the threshold were                 found


,Name,Prediction,WT,Mutant,Score
ID,,,,,
met__L_e,L-Methionine,Elevated,"[-10.0, 0.0]","[-0.0, 0.0]",1.00
cys__L_e,L-Cysteine,Reduced,"[-10.0, 40.0]","[-10.0, 30.0]",0.25


In [29]:
findBiomarkers(M,rxnsOfInterest,IEMdict_rxns['Homocystinuria'],threshold=0.1)

Interpreting mode as IEMrxn
Modifications will be performed on the following reactions:
                                            Description                                          Reaction
CYSTS   Cystathionine beta synthase                      hcys__L_c + ser__L_c --> cyst__L_c + h2o_c      
MTHFR3  5 10 methylenetetrahydrofolatereductase  NADPH   2.0 h_c + mlthf_c + nadph_c --> 5mthf_c + nadp_c
METS    Methionine synthase                              5mthf_c + hcys__L_c --> h_c + met__L_c + thf_c  




2 low confidence biomarkers with scores below the threshold were                 found


,Name,Prediction,WT,Mutant,Score
ID,,,,,
met__L_e,L-Methionine,H.C. Elevated,"[-10.0, -1.0]","[-0.0, 0.0]",1.00
cys__L_e,L-Cysteine,Reduced,"[-10.0, 40.0]","[-10.0, 30.0]",0.25


### Show that using influx of -1 instead of -10 results in different predictions for Figure 3
With medium influx of -1, the homocystinuria prediction in the WT becomes [-1,-1] which is not an interval as it was drawn in Shlomi et al. Figure 3B.

In [31]:
model = M.copy()

# Note: exchange reactions are pseudo-reactions of the form: 'X_e <=> '
exchanges = [ rxn for rxn in model.reactions if len(rxn.products) == 0]

# all inward fluxes to -1
# all outward fluxes to 1000
for rxn in exchanges:
    rxn.lower_bound = -1
for rxn in exchanges:
        rxn.upper_bound = 1000
    
findBiomarkers(model,rxnsOfInterest,IEMdict_rxns['Homocystinuria'],threshold=0.1)

Interpreting mode as IEMrxn
Modifications will be performed on the following reactions:
                                            Description                                          Reaction
CYSTS   Cystathionine beta synthase                      hcys__L_c + ser__L_c --> cyst__L_c + h2o_c      
MTHFR3  5 10 methylenetetrahydrofolatereductase  NADPH   2.0 h_c + mlthf_c + nadph_c --> 5mthf_c + nadp_c
METS    Methionine synthase                              5mthf_c + hcys__L_c --> h_c + met__L_c + thf_c  



Removed glu__L_e because it has an equal number of contradictory predictions.
Removed ala__L_e because it has an equal number of contradictory predictions.

6 low confidence biomarkers with scores below the threshold were                 found


,Name,Prediction,WT,Mutant,Score
ID,,,,,
met__L_e,L-Methionine,H.C. Elevated,"[-1.0, -1.0]","[0.0, -0.0]",1.00
cys__L_e,L-Cysteine,Reduced,"[-1.0, 4.0]","[-1.0, 3.0]",0.25
